In [1]:
#소벨 에지 검출(Sobel 함수 사용)하기
import cv2 as cv

img = cv.imread('soccer.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
gray_x = cv.Sobel(gray, cv.CV_32F, 1,0,ksize=3) #(결과 영상을 32비트 실수 맵에 저장하라고 지시, x방향 연산자 사용지시, 3*3크기)
gray_y = cv.Sobel(gray, cv.CV_32F, 0,1,ksize=3) #(결과 영상을 32비트 실수 맵에 저장하라고 지시, y방향 연산자 사용지시, 3*3크기)

sobel_x = cv.convertScaleAbs(gray_x) #절댓값을 취해 양수 영상으로 전환
sobel_y = cv.convertScaleAbs(gray_y)

edge_strength = cv.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0) #에지 강도 계산

cv.imshow('Original', gray)
cv.imshow('sobelx', sobel_x)
cv.imshow('sobely', sobel_y)
cv.imshow('edge strength', edge_strength)

cv.waitKey()
cv.destroyAllWindows()

In [2]:
#캐니 에지 실험하기
img = cv.imread('soccer.jpg')
img = cv.resize(img, dsize=(0,0), fx=0.25, fy=0.25)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

canny1 = cv.Canny(gray,50,150)
canny2 = cv.Canny(gray,100,200)

cv.imshow('Original', gray)
cv.imshow('Canny1', canny1)
cv.imshow('Canny2', canny2)

cv.waitKey()
cv.destroyAllWindows()

In [4]:
import cv2

def empty(pos):
    pass
name = 'Trackbar'
cv2.namedWindow(name)
cv2.createTrackbar('threshold1', name, 0, 255, empty) # min value
cv2.createTrackbar('threshold2', name, 0, 255, empty) # max value

while True:
    threshold1 = cv2.getTrackbarPos('threshold1', name)
    threshold2 = cv2.getTrackbarPos('threshold2', name)
    
    canny = cv2.Canny(img, threshold1, threshold2)
    
    cv2.imshow('img', img)
    cv2.imshow(name, canny)

    if cv2.waitKey(1) == ord('q'):
        break
    
cv2.destroyAllWindows()

In [5]:
#에지 맵에서 경계선 찾기
import numpy as np
canny = cv.Canny(gray, 100, 200)
#cv.RETR_LIST로 설정해 맨 바깥쪽 경계선만 찾도록 지시. cv.CHAIN_APPROX_NONE 사용 시 모든 점을 기록.
#cv.CHAIN_APPROX_SIMPLE로 설정 시 직선에 대해서는 양 끝점만 기록.
#cv.CHAIN_APPROX_TC89_L1 or cv.CHAIN_APPROX_TC89_KCOS는 Tch_Chin 알고리즘으로 굴곡이 심한 점을 찾아 그들만 기록
contour, hierarchy = cv.findContours(canny, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

lcontour = []
for i in range(len(contour)):
    if contour[i].shape[0] >100:
        lcontour.append(contour[i])
#drawContours(영상, 경계선, 경계선 설정, 색, 두께)
cv.drawContours(img, lcontour, -1, (0,255,0), 3)

cv.imshow('Original', img)
cv.imshow('Canny', canny)

cv.waitKey()
cv.destroyAllWindows()

In [6]:
#허프 변환을 이용해 사과 검출하기
img = cv.imread('apples.jpg')
img = cv.resize(img, dsize=(0,0), fx=0.25, fy=0.25)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#cv.HOGGH_GRADIENT는 에지 방향 정보를 추가로 사용하는 방법
#3번째 인수는 원 사이의 최소 거리를 지정하는데 작을수록 많은 원이 검출, 5번째 인수는 캐니 에지 알고리즘이 사용하는 T
#6번째 인수는 비최대 억제를 적용할 때 쓰는 임곗값, 7,8번째 인수는 원의 최소와 최대 반지름을 지정.
apples = cv.HoughCircles(gray, cv.HOUGH_GRADIENT,1,200,param1=150, param2=20,minRadius=50, maxRadius=120)
for i in apples[0]:
    cv.circle(img, (int(i[0]), int(i[1])), int(i[2]), (255,0,0),2)
cv.imshow('Apple', img)
cv.waitKey()
cv.destroyAllWindows()

In [7]:
!pip install scikit-image

In [8]:
#SLIC 알고리즘으로 입력 영상을 슈퍼 화소 분할하기
import skimage
import numpy as np
import cv2 as cv

img = skimage.data.coffee()
cv.imshow('Coffee', cv.cvtColor(img, cv.COLOR_RGB2BGR))
#slic함수로 슈퍼 화소 분할 수행, compactness 인수는 슈퍼 화소의 모양을 조절.
slic1 = skimage.segmentation.slic(img, compactness=20, n_segments=600)
sp_img1 = skimage.segmentation.mark_boundaries(img, slic1)
#0~255 사이로 바꾸고 uint8형으로 변환.
sp_img1 = np.uint8(sp_img1 * 255.0)

slic2 = skimage.segmentation.slic(img, compactness=40, n_segments=600)
sp_img2 = skimage.segmentation.mark_boundaries(img, slic2)
sp_img2 = np.uint8(sp_img2 * 255.0)

cv.imshow('Super pixels (compact 20)', cv.cvtColor(sp_img1, cv.COLOR_RGB2BGR))
cv.imshow('Super pixels (compact 40)', cv.cvtColor(sp_img2, cv.COLOR_RGB2BGR))

cv.waitKey()
cv.destroyAllWindows()

In [9]:
!pip install -U scikit-image

In [11]:
#정규화 절단 알고리즘으로 영역 분할하기
import skimage
import numpy as np
import cv2 as cv
import time

coffee=skimage.data.coffee()  # coffee 영상을 읽어

start=time.time()  # 분할하는 데 걸리는 시간을 측정해 출력
slic=skimage.segmentation.slic(coffee,compactness=20,n_segments=600,start_label=1)
g=skimage.graph.rag_mean_color(coffee,slic,mode='similarity') 
ncut=skimage.graph.cut_normalized(slic,g)	# 정규화 절단
print(coffee.shape,' Coffee 영상을 분할하는데 ',time.time()-start,'초 소요')

marking=skimage.segmentation.mark_boundaries(coffee,ncut)  # 원래 영상인 coffee에 영역 분할 정보를 담은 ncut 맵을 이용하여 영역 경계를 표 시하고 marking 객체에 저장
ncut_coffee=np.uint8(marking*255.0)  # 0~1 사이의 실수를 가진 marking을 0~255 사이의 uint8 형으로 변환

cv.imshow('Normalized cut',cv.cvtColor(ncut_coffee,cv.COLOR_RGB2BGR))  # RGB로 표현된 분할 영상을 BGR로 변환하여 윈도우에 디스플레이

cv.waitKey()
cv.destroyAllWindows()

(400, 600, 3)  Coffee 영상을 분할하는데  5.620427131652832 초 소요


In [12]:
#영역 개수 확인
np.unique(ncut)
np.unique(ncut).size

27

In [14]:
import cv2 as cv 
import numpy as np

img=cv.imread('soccer.jpg')	# 영상 읽기
img_show=np.copy(img)		# 붓 칠을 디스플레이할 목적의 영상

mask=np.zeros((img.shape[0],img.shape[1]),np.uint8) 
mask[:,:]=cv.GC_PR_BGD		# 모든 화소를 배경일 것 같음으로 초기화

BrushSiz=9				# 붓의 크기
LColor,RColor=(255,0,0),(0,0,255)	# 파란색(물체)과 빨간색(배경)

def painting(event,x,y,flags,param):
    if event==cv.EVENT_LBUTTONDOWN:   
        cv.circle(img_show,(x,y),BrushSiz,LColor,-1)	# 왼쪽 버튼 클릭하면 파란색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_FGD,-1)
    elif event==cv.EVENT_RBUTTONDOWN: 
        cv.circle(img_show,(x,y),BrushSiz,RColor,-1)	# 오른쪽 버튼 클릭하면 빨간색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_BGD,-1)
    elif event==cv.EVENT_MOUSEMOVE and flags==cv.EVENT_FLAG_LBUTTON:
        cv.circle(img_show,(x,y),BrushSiz,LColor,-1)# 왼쪽 버튼 클릭하고 이동하면 파란색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_FGD,-1)
    elif event==cv.EVENT_MOUSEMOVE and flags==cv.EVENT_FLAG_RBUTTON:
        cv.circle(img_show,(x,y),BrushSiz,RColor,-1)	# 오른쪽 버튼 클릭하고 이동하면 빨간색
        cv.circle(mask,(x,y),BrushSiz,cv.GC_BGD,-1)

    cv.imshow('Painting',img_show)
    
cv.namedWindow('Painting')
cv.setMouseCallback('Painting',painting)

while(True):				# 붓 칠을 끝내려면 'q' 키를 누름
    if cv.waitKey(1)==ord('q'): 
        break

# 여기부터 GrabCut 적용하는 코드
background=np.zeros((1,65),np.float64)	# 배경 히스토그램 0으로 초기화
foreground=np.zeros((1,65),np.float64)	# 물체 히스토그램 0으로 초기화

cv.grabCut(img,mask,None,background,foreground,5,cv.GC_INIT_WITH_MASK)
mask2=np.where((mask==cv.GC_BGD)|(mask==cv.GC_PR_BGD),0,1).astype('uint8')
grab=img*mask2[:,:,np.newaxis]
cv.imshow('Grab cut image',grab)  

cv.waitKey()
cv.destroyAllWindows()

In [15]:
import skimage
import numpy as np
import cv2 as cv

orig=skimage.data.horse()
img=255-np.uint8(orig)*255
cv.imshow('Horse',img)

contours,hierarchy=cv.findContours(img,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_NONE)

img2=cv.cvtColor(img,cv.COLOR_GRAY2BGR)		# 컬러 디스플레이용 영상
cv.drawContours(img2,contours,-1,(255,0,255),2)
cv.imshow('Horse with contour',img2)

contour=contours[0]

m=cv.moments(contour)				# 몇 가지 특징 
area=cv.contourArea(contour)
cx,cy=m['m10']/m['m00'],m['m01']/m['m00']
perimeter=cv.arcLength(contour,True)
roundness=(4.0*np.pi*area)/(perimeter*perimeter)
print('면적=',area,'\n중점=(',cx,',',cy,')','\n둘레=',perimeter,'\n둥근 정도=',roundness)

img3=cv.cvtColor(img,cv.COLOR_GRAY2BGR)		# 컬러 디스플레이용 영상

contour_approx=cv.approxPolyDP(contour,8,True)	# 직선 근사
cv.drawContours(img3,[contour_approx],-1,(0,255,0),2)

hull=cv.convexHull(contour)			# 볼록 헐
hull=hull.reshape(1,hull.shape[0],hull.shape[2])
cv.drawContours(img3,hull,-1,(0,0,255),2)

cv.imshow('Horse with line segments and convex hull',img3)

cv.waitKey()
cv.destroyAllWindows()

면적= 42390.0 
중점=( 187.72464024534088 , 144.43640402610677 ) 
둘레= 2296.7291333675385 
둥근 정도= 0.1009842680321435
